## Introduction to Routing Analysis using Batfish

Network engineers routinely need to validate routing and forwarding in the network. They often do that by connecting to multiple network devices and executing a series of `show` or `traceroute` commands. This distributed debugging task can be highly complex even in a moderately-sized network. Batfish makes this task extremely simple by providing an easy-to-query, centralized view of routing and forwarding in the network. 

In this notebook, we will look at how you can extract routing-related information from Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-route-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

 ### Initializing the Network and Snapshot

In [2]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "../test_rigs/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can download/view devices' configuration files [here](https://github.com/batfish/batfish/tree/master/test_rigs/example).

![example-network](https://raw.githubusercontent.com/batfish/batfish/master/test_rigs/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

### Running Traceroutes
`traceroute` is a work horse for debugging routing. Batfish provides a simple way to run traceroute from any source to any destination using any type of packet. In addition, unlike the live-network traceroute, it shows all possible paths that a flow might take and routing entries that cause the path to be taken.

In code below, `.frame()` returns a Pandas Dataframe. Go [here](https://pandas.pydata.org/pandas-docs/stable/tutorials.html) to learn more about Pandas.

In [3]:
# Run a traceroute from as2border1 to host1, while also controlling the source IP and destination protocol
tr_answer = bfq.traceroute(traceStart="as2border1", dst="host1", dstProtocol="dns", srcIpSpace="10.12.11.2").answer()

# Display the results in a friendly manner
display_html(tr_answer.frame())

,node,dstIp,flow,numPaths,results,paths
0,as2border1,2.128.0.101,as2border1->[10.12.11.2:0->2.128.0.101:53 proto: UDP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,4,['ACCEPTED'],"1 as2border1:GigabitEthernet1/0 -> as2core1:GigabitEthernet0/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.11.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.11.22 as2core1:GigabitEthernet2/0 -> as2dist1:GigabitEthernet0/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.23.11.33 as2dist1:GigabitEthernet2/0 -> as2dept1:GigabitEthernet0/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.34.101.44 as2dept1:GigabitEthernet2/0 -> host1:eth0 Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:nullACCEPTED1 as2border1:GigabitEthernet1/0 -> as2core1:GigabitEthernet0/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.11.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.11.22 as2core1:GigabitEthernet3/0 -> as2dist2:GigabitEthernet1/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.23.12.33 as2dist2:GigabitEthernet2/0 -> as2dept1:GigabitEthernet1/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.34.201.44 as2dept1:GigabitEthernet2/0 -> host1:eth0 Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:nullACCEPTED1 as2border1:GigabitEthernet2/0 -> as2core2:GigabitEthernet1/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.12.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.12.22 as2core2:GigabitEthernet2/0 -> as2dist2:GigabitEthernet0/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.23.22.33 as2dist2:GigabitEthernet2/0 -> as2dept1:GigabitEthernet1/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.34.201.44 as2dept1:GigabitEthernet2/0 -> host1:eth0 Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:nullACCEPTED1 as2border1:GigabitEthernet2/0 -> as2core2:GigabitEthernet1/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.12.12.2 BgpRoute<2.128.0.0/24,nhip:2.34.201.4,nhint:dynamic>_fnhip:2.12.12.22 as2core2:GigabitEthernet3/0 -> as2dist1:GigabitEthernet1/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.23.21.33 as2dist1:GigabitEthernet2/0 -> as2dept1:GigabitEthernet0/0 Route(s): BgpRoute<2.128.0.0/24,nhip:2.34.101.4,nhint:dynamic>_fnhip:2.34.101.44 as2dept1:GigabitEthernet2/0 -> host1:eth0 Route(s): ConnectedRoute<2.128.0.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet2/0>_fnhip:nullACCEPTED"


### Viewing the (Logical)  Topology
Beyond `traceroute` which shows end-to-end paths, Batfish can provide more detailed information as well. For instance, it can produce the Layer 3 (IP) topology of the snapshot.

In [4]:
# Get all 'layer3' neighbors of 'as2border2' 
answer = bfq.neighbors(node1Regex="as2border2", neighborTypes=["layer3"]).answer()['answerElements'][0]

# Relevant information is present under the key 'layer3Neighbors' of the answer
pd.DataFrame(answer['layer3Neighbors'])

,node1,node1interface,node2,node2interface
0,as2border2,GigabitEthernet0/0,as3border1,GigabitEthernet1/0
1,as2border2,GigabitEthernet1/0,as2core2,GigabitEthernet0/0
2,as2border2,GigabitEthernet2/0,as2core1,GigabitEthernet1/0


Batfish can also provide a VRF-specific view of the topology.

In [5]:
# Show the VRF of each interface on as2border2
interface_vrf = bfq.interfaceProperties(nodeRegex="as2border2", propertySpec="vrf|interface").answer().frame()
interface_vrf

,interface,vrf
0,as2border2:GigabitEthernet1/0,default
1,as2border2:GigabitEthernet2/0,default
2,as2border2:Ethernet0/0,default
3,as2border2:Loopback0,default
4,as2border2:GigabitEthernet0/0,default


In the example network, no VRFs have been configured, so all interfaces are part of the **default** VRF. This is also referred to as the **global VRF** or **global table** by some vendors and network engineers.

### Viewing Routing Tables and Debugging Connectivity
To further understand routing, engineers may want to look at the routing tables directly. Batfish makes these tables accessible for all nodes and VRFs.

In [6]:
# Get the routing table for the 'default' VRF on 'as2border2'
routing_table = bfq.routes(nodeRegex="as2border2", vrfRegex="default").answer().frame()
routing_table

,Node,VRF,Network,Protocol,Tag,NextHopIp,NextHop,AdminDistance,Metric
0,as2border2,default,2.1.3.1/32,ospf,-1,2.12.22.2,as2core2,110,3
1,as2border2,default,2.1.3.1/32,ospf,-1,2.12.21.2,as2core1,110,3
2,as2border2,default,10.23.21.2/32,local,-1,AUTO/NONE(-1l),None,0,0
3,as2border2,default,2.1.2.2/32,ospf,-1,2.12.22.2,as2core2,110,2
4,as2border2,default,2.12.12.0/24,ospf,-1,2.12.22.2,as2core2,110,2
5,as2border2,default,3.0.1.0/24,bgp,-1,10.23.21.3,as3border1,20,50
6,as2border2,default,2.23.21.0/24,ospf,-1,2.12.22.2,as2core2,110,2
7,as2border2,default,2.12.21.1/32,local,-1,AUTO/NONE(-1l),None,0,0
8,as2border2,default,2.128.0.0/16,aggregate,-1,AUTO/NONE(-1l),None,200,0
9,as2border2,default,1.0.2.0/24,ibgp,-1,10.12.11.1,as1border1,200,50


To debug a connectivity issue, network engineers often need to analyze the routing entries for a prefix tables at multiple devices. The commands below show how to accomplish this task, using the prefix **1.0.2.0/24** as an example.

In [7]:
# Get all routes at all nodes
all_routes = bfq.routes().answer().frame()

# Get routes for '1.0.2.0/24'
filtered_routes = all_routes[all_routes["Network"] == "1.0.2.0/24"]

# Get the unique node names (using Pandas) and sort the list
print(sorted(filtered_routes["Node"].unique()))

['as1border1', 'as1border2', 'as1core1', 'as2border1', 'as2border2', 'as2core1', 'as2core2', 'as2dept1', 'as2dist1', 'as2dist2', 'as3border1', 'as3border2', 'as3core1']


Listing which routers have routes to a given prefix is useful, but we may also want to know which routers do **not** have the prefix. We can easily accomplish that as well using the Pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) and [filter](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.filter.html) functions.

In [8]:
# Group all routes by Node and filter for those that don't have '1.0.2.0/24'
filtered_groups = all_routes.groupby('Node').filter(lambda x: all(x['Network'] != '1.0.2.0/24'))

# Get the unique node names and sort the list
print(sorted(filtered_groups["Node"].unique()))

['host1', 'host2']


In this case, all of the routers happen to have this prefix. Only the 2 servers 'host1' and 'host2' do not have it.

This notebook showed you

1. How to run `traceroute` using Batfish 
2. How to retrieve information about the routing topology of the network
3. How to retrieve and analyze routing tables to debug connectivity to a prefix

These are foundational tasks for routing analysis. Future notebooks will dive into more advanced route analysis. Stay tuned.

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more or send feedback.